# 颜色转移算法

我对颜色转移的实现（宽松地）是基于 Reinhard等人（2001年）[*在“图像之间的颜色转移”*](http://www.thegooch.org/Publications/PDFs/ColorTransfer.pdf)。

Reinhard及其同事在本文中证明，通过利用L * a * b *颜色空间以及每个L *，a *和b *通道的均值和标准差，可以在两个图像之间转移颜色。

该算法如下所示：

- **步骤1：** 输入`source`和`target`。源图像包含您要`target`图像模仿的色彩空间。在此页面顶部的图中，左侧的日落图像是my `source`，中间的图像是my `target`，右侧的图像是`source`应用到的色彩空间`target`。
- **步骤2：** 将`source`和`target`图像都转换为L * a * b *颜色空间。L * a * b *颜色空间模拟了感知均匀性，其中颜色值的微小变化也应该在颜色重要性上产生相对相等的变化。与标准RGB颜色空间相比，L * a * b *颜色空间在模仿人类如何解释颜色方面做得更好，并且您将看到，它在颜色转移方面效果很好。
- **步骤3：** 分别为`source`和分配频道`target`。
- **步骤4：** 为`source`和`target`图像计算每个L * a * b *通道的平均值和标准差。
- **步骤5：**`target`从`target`通道中 减去图像的L * a * b *通道的平均值。
- **步骤6：**`target` 按标准偏差`target`除以的标准偏差`source`乘以通道的比例来 缩放`target` 通道。
- **步骤7：** 为加上L * a * b *通道的均值`source`。
- **步骤8：** 裁剪掉*[0，255]*范围之外的所有值。（**注意：** 此步骤不是原始文章的一部分。由于OpenCV如何处理色彩空间转换，所以我添加了此步骤。如果要以其他语言/库实现此算法，则必须执行色彩空间转换您自己，或了解进行转换的库如何工作）。
- **第9步：** 将频道合并在一起。
- **步骤10：** 从L * a * b *空间转换回RGB颜色空间。

In [1]:
import cv2
import numpy as np

def image_stats(image):
    (l, a, b) = cv2.split(image)
    (lMean, lStd) = (l.mean(), l.std())
    (aMean, aStd) = (a.mean(), a.std())
    (bMean, bStd) = (b.mean(), b.std())
    
    return (lMean, lStd, aMean, aStd, bMean, bStd)

def color_transfer(source, target):
    oldSource = cv2.imread(source)
    oldTarget = cv2.imread(target)
    
    # 将图像从RGB转换为L * ab *颜色空间，即
    # 确保使用浮点数据类型（注意：OpenCV
    # 期望浮点数为32位，因此请使用浮点数而不是64位）
    source = cv2.cvtColor(oldSource, cv2.COLOR_BGR2LAB).astype("float32")
    target = cv2.cvtColor(oldTarget, cv2.COLOR_BGR2LAB).astype("float32")
    
    # 计算源图像和目标图像的颜色统计信息
    (lMeanSrc, lStdSrc, aMeanSrc, aStdSrc, bMeanSrc, bStdSrc) = image_stats(source)
    (lMeanTar, lStdTar, aMeanTar, aStdTar, bMeanTar, bStdTar) = image_stats(target)
    
    (l, a, b) = cv2.split(target)
    l -= lMeanTar
    a -= aMeanTar
    b -= bMeanTar
    
    l = (lStdSrc / lStdTar) * l
    a = (aStdSrc / aStdTar) * a
    b = (bStdSrc / bStdTar) * b

    l += lMeanSrc
    a += aMeanSrc
    b += bMeanSrc
    
    # 将超出[0，255]范围的像素值改为0或255
    l = np.clip(l, 0, 255)
    a = np.clip(a, 0, 255)
    b = np.clip(b, 0, 255)
    
    # 将通道合并在一起并转换回RGB颜色
    # 空间，请确保使用8位无符号整数数据类型
    transfer = cv2.merge([l, a, b])
    transfer = cv2.cvtColor(transfer.astype("uint8"), cv2.COLOR_LAB2BGR)

    cv2.imshow('Source', oldSource)
    cv2.imshow('Target', oldTarget)
    cv2.imshow('Transfer', transfer)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    return transfer

In [2]:
transfer = color_transfer('ocean_sunset.jpg', 'ocean_day.jpg')

In [3]:
transfer = color_transfer('ocean_day.jpg', 'ocean_sunset.jpg')

In [4]:
transfer = color_transfer('autumn.jpg', 'fallingwater.jpg')

In [5]:
transfer = color_transfer('fallingwater.jpg', 'autumn.jpg')

In [6]:
transfer = color_transfer('woods.jpg', 'storm.jpg')

In [7]:
transfer = color_transfer('storm.jpg', 'woods.jpg')